In [1]:
from __future__ import division
from mnist import MNIST
from sklearn import tree
import numpy as np
from sklearn.metrics import classification_report,log_loss,accuracy_score,roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from time import time
import pandas as pd
from datetime import datetime
from openpyxl import load_workbook
import re
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from mlxtend.data import loadlocal_mnist
import timeit
import timeit 
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import confusion_matrix
from numpy import argmax

import scipy.io
mat = scipy.io.loadmat('mnist-with-awgn.mat')
images_train_o, labels_train_o = mat['train_x'], mat['train_y']
images_test_o, labels_test_o = mat['test_x'], mat['test_y']
labels_test_o1 = np.array(labels_test_o)
labels_train_o1  = np.array(labels_train_o)
images_train_o = np.array(images_train_o)
images_test_o = np.array(images_test_o)
labels_test_o2=np.zeros((10000,1))
labels_train_o2=np.zeros((60000,1))
#print(images_test.shape)
for i in range(0,10000):
    labels_test_o2[i]=np.argmax(labels_test_o[i])
for i in range(0,60000):
    labels_train_o2[i]=np.argmax(labels_train_o[i])    
    
print(labels_test_o2)

/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


[[ 2.]
 [ 1.]
 [ 9.]
 ..., 
 [ 0.]
 [ 9.]
 [ 5.]]


In [2]:
excel_file_name = 'mnist-with-awgn' + str(datetime.now().strftime('%Y%m%d%H%M%S')) + '.xlsx'
excel_writer = pd.ExcelWriter(excel_file_name, engine='openpyxl')
df = pd.DataFrame(columns=['Examples','Accuracy','Normalized Accuracy', 
                     'Time_taken_train','Time_taken_test',
                     'Negative Log loss','Macro_auc','Micro_auc'])
df.to_excel(excel_writer, sheet_name='demo_sheet')

excel_writer.save()


/usr/lib/python3/dist-packages/pandas/io/excel.py:687: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])


In [3]:
def classifaction_report_to_dataframe(report):
    report_data = []
    lines = report.split('\n')
    for line in lines[2:-3]:
        row = {}
        row_data = line.split('      ')
        print(line)
        if len(row_data)>5:
            row['class'] = row_data[1]
            row['precision'] = float(row_data[2])
            row['recall'] = float(row_data[3])
            row['f1_score'] = float(row_data[4])
            row['support'] = float(row_data[5])
            report_data.append(row)
    for line in lines[-2:-1]:
        row = {}
        p = re.compile("weighted avg")
        m = p.search(line)  
        print(line)
        row['class'] = m.group(0)
        p = re.compile("\d+.?\d+")
        row_data = p.findall(line)
        row['precision'] = float(row_data[0])
        row['recall'] = float(row_data[1])
        row['f1_score'] = float(row_data[2])
        row['support'] = float(row_data[3])
        report_data.append(row)
        
    dataframe = pd.DataFrame.from_dict(report_data)
    return dataframe

In [4]:
for t in[60000]:
    print('For t=',t)
    images_train=images_train_o
    labels_train=labels_train_o2
    images_test=images_test_o
    labels_test=labels_test_o2
    svm = LinearSVC(dual=True)
    classifier= CalibratedClassifierCV(svm)
    start=time()
    print(labels_train.shape)
    classifier.fit(images_train,labels_train)
    train_time=time()-start
    start=time()
    predictions=classifier.predict(images_test)
    test_time=time()-start
    accuracy=accuracy_score(predictions,labels_test,normalize=False)
    normalized_accuracy=accuracy_score(predictions,labels_test,normalize=True)
    log_loss_prediction=classifier.predict_proba(images_test)
    log_loss_val=log_loss(labels_test,log_loss_prediction)
    classification_repo=classification_report(labels_test,predictions)

    labels_test_one_hot=label_binarize(labels_test,classes=[0,1,2,3,4,5,6,7,8,9])
    predictions_one_hot=label_binarize(predictions,classes=[0,1,2,3,4,5,6,7,8,9])
    micro_auc=roc_auc_score(labels_test_one_hot,predictions_one_hot,average="micro")
    macro_auc=roc_auc_score(labels_test_one_hot,predictions_one_hot,average="macro")

    #print(micro_auc,macro_auc)
    cm = confusion_matrix(labels_test,predictions)
    cm_df = pd.DataFrame(data=cm)
    df_data = [t,accuracy,normalized_accuracy,train_time,test_time,log_loss_val,macro_auc,micro_auc]
    df.loc[len(df)] = df_data   

    book = load_workbook(excel_file_name)
    with pd.ExcelWriter(excel_file_name, engine='openpyxl') as excel_writer:
        excel_writer.book = book
        df_classification_report = classifaction_report_to_dataframe(classification_repo)
        df_classification_report.to_excel(excel_writer, sheet_name='Linearsvm_cr'+str(t))

        cm_df.to_excel(excel_writer, sheet_name='mnist-with-awg'+str(t),
                       startrow=len(df_classification_report)+5)

        excel_writer.save()

book = load_workbook(excel_file_name)
with pd.ExcelWriter(excel_file_name, engine='openpyxl') as excel_writer:
    excel_writer.book = book
    df.to_excel(excel_writer, sheet_name='mnist-with-awg')
    excel_writer.save()



For t= 60000
(60000, 1)


/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWar

         0.0       0.92      0.96      0.94       980
         1.0       0.90      0.96      0.93      1135
         2.0       0.88      0.81      0.85      1032
         3.0       0.84      0.87      0.85      1010
         4.0       0.85      0.87      0.86       982
         5.0       0.80      0.75      0.78       892
         6.0       0.89      0.92      0.91       958
         7.0       0.87      0.87      0.87      1028
         8.0       0.82      0.78      0.80       974
         9.0       0.82      0.80      0.81      1009

   micro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000


/usr/lib/python3/dist-packages/pandas/io/excel.py:687: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])
